GROUP 13

In [1]:
import pandas as pd
import numpy as np
import glob 
import os 
from scipy.signal import find_peaks
from scipy.signal import find_peaks



master_df = pd.read_csv(r'C:\Users\DELL\Desktop\vocational traning\project 1\master_data (1).csv', header = None)
print (master_df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\DELL\\Desktop\\vocational traning\\project 1\\master_data (1).csv'

In [ ]:

subject_files = sorted(glob.glob(r'C:\Users\DELL\Desktop\vocational traning\project 1\subject_*.csv'))

subject_dfs = []

for file in subject_files:
    df = pd.read_csv(file)
    
    subject_id = int(os.path.basename(file).split("_")[1].split(".")[0]) #subject id extraction from file 
    
    df['subject_id'] = subject_id  #this make sure thata sub id is present 
    subject_dfs.append(df)


In [ ]:
all_subjects_df = pd.concat(subject_dfs, ignore_index=True)
print(all_subjects_df.head())


       time  ppg_signal  subject_id
0  0.000000   -0.005162           1
1  0.010002    0.115503           1
2  0.020003    0.239821           1
3  0.030005    0.339529           1
4  0.040007    0.472772           1


In [ ]:

master_df = pd.read_csv(r'C:\Users\DELL\Desktop\vocational traning\project 1\master_data (1).csv')

# Ensure both subject_id columns are of the same type (int)
master_df['subject_id'] = master_df['subject_id'].astype(int)
all_subjects_df['subject_id'] = all_subjects_df['subject_id'].astype(int)

merged_df = pd.merge(all_subjects_df, master_df, on="subject_id", how="left")
print(merged_df.head())

       time  ppg_signal  subject_id         hr        hrv
0  0.000000   -0.005162           1  95.579696  66.052442
1  0.010002    0.115503           1  95.579696  66.052442
2  0.020003    0.239821           1  95.579696  66.052442
3  0.030005    0.339529           1  95.579696  66.052442
4  0.040007    0.472772           1  95.579696  66.052442


In [ ]:
# Saving 
merged_df.to_csv(r'C:\Users\DELL\Desktop\vocational traning\project 1\merged_ppg_data.csv', index=False)



In [ ]:
# mean
mean_val = merged_df["ppg_signal"].mean()
print(mean_val)


0.0014826874985018348


In [ ]:
variance_val = merged_df.groupby("subject_id")["ppg_signal"].var()
print(variance_val)

subject_id
1     0.224945
2     0.225262
3     0.225294
4     0.225213
5     0.225614
6     0.225374
7     0.225453
8     0.225395
9     0.224770
10    0.225635
11    0.225579
12    0.225418
13    0.224923
14    0.225644
15    0.225434
16    0.225176
17    0.224751
18    0.225300
19    0.224341
20    0.225228
Name: ppg_signal, dtype: float64


In [ ]:

#peak count 
peak_count =[]
for subject_id in merged_df['subject_id'].unique():
    sub_data = merged_df[merged_df['subject_id'] == subject_id]
    signal = sub_data['ppg_signal'].values
    peaks, _ = find_peaks(signal)
    peak_count.append(len(peaks))
    
peak_count_df = pd.DataFrame(peak_count)
print(peak_count_df.head())

 


     0
0  411
1  440
2  531
3  438
4  394


In [ ]:
#peak to peak interval

rr_interval = []
rr_interval_stats = []

for subject_id in merged_df['subject_id'].unique():
    
    subject_data = merged_df[merged_df['subject_id'] == subject_id]
    
    signal = subject_data['ppg_signal'].values
    time = subject_data['time'].values
    

    peaks, _ = find_peaks(signal)  


    rr_intervals = np.diff(time[peaks]) * 1000  

    rr_interval_stats.append({
        'subject_id': subject_id,
        'avg_rr_interval_ms': np.mean(rr_intervals) if len(rr_intervals) > 0 else np.nan,
        'std_rr_interval_ms': np.std(rr_intervals) if len(rr_intervals) > 0 else np.nan
    })


rr_interval_df = pd.DataFrame(rr_interval_stats)

print(rr_interval_df.head())



   subject_id  avg_rr_interval_ms  std_rr_interval_ms
0           1          146.048732          130.102519
1          10          136.309734          137.221061
2          11          112.792384          132.876321
3          12          136.086754          135.478672
4          13          152.366361          133.995159


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Merge rr_interval_df with master_df to get hr as target
features_df = pd.merge(rr_interval_df, master_df[['subject_id', 'hr']], on='subject_id')

# Features (X) and Target (y)
X = features_df.drop(columns=['subject_id', 'hr'])
y = features_df['hr']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# traing 
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)


LinearRegression()

In [ ]:
# Make predictions
y_pred = lr_model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

# Display metrics
print(f"Linear Regression Performance:\nMAE: {mae:.2f}\nRMSE: {rmse:.2f}\nR² Score: {r2:.2f}")


Linear Regression Performance:
MAE: 1.80
RMSE: 2.55
R² Score: 0.97


In [ ]:


%pip install nbformat



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import plotly.express as px

if 'merged_df' in globals():
	fig = px.line(x=merged_df['time'], y=merged_df['ppg_signal'], title='PPG Signal Over Time')
	fig.show()
else:
	print("Please run the cell that defines 'merged_df' before running this cell.")

Please run the cell that defines 'merged_df' before running this cell.


In [ ]:
#loading subject 1 data

subject_1_df = pd.read_csv(r'C:\Users\DELL\Desktop\vocational traning\project 1\subject_1.csv')
print(subject_1_df.head())

       time  ppg_signal  subject_id
0  0.000000   -0.005162           1
1  0.010002    0.115503           1
2  0.020003    0.239821           1
3  0.030005    0.339529           1
4  0.040007    0.472772           1


In [ ]:
#ploting subject 1 data 

fig = px.line(x=subject_1_df['time'], y=subject_1_df['ppg_signal'], title='Subject 1 PPG Signal Over Time')
fig.show()

In [ ]:
#finding subject 1 mean 

subject_1_mean = subject_1_df['ppg_signal'].mean()
print(subject_1_mean)

0.00178060788539396


In [ ]:
#subject 1  median 

subjec_1_median = subject_1_df['ppg_signal'].median()
print(subjec_1_median)

0.001270761080362


In [ ]:
#subject 1 variance 

subject_1_variance = subject_1_df['ppg_signal'].var()
print(subject_1_variance)


0.22494504871165374


In [ ]:


subject_1_peak_count = []
for subject_id in subject_1_df['subject_id'].unique():
    sub_data = subject_1_df[subject_1_df['subject_id'] == subject_id]
    signal = sub_data['ppg_signal'].values
    peaks, _ = find_peaks(signal)
    subject_1_peak_count.append(len(peaks))
    
subject_1_peak_count_df = pd.DataFrame(subject_1_peak_count)
print(subject_1_peak_count_df.head())

     0
0  411


In [ ]:


subject_1_rr_interval = []
subject_1_rr_interval_stats = []

for subject_id in subject_1_df['subject_id'].unique():
    
    subject_data = subject_1_df[subject_1_df['subject_id'] == subject_id]
    
    signal = subject_data['ppg_signal'].values
    time = subject_data['time'].values
    

    peaks, _ = find_peaks(signal)  


    rr_intervals = np.diff(time[peaks]) * 1000  

    subject_1_rr_interval_stats.append({
        'subject_id': subject_id,
        'avg_rr_interval_ms': np.mean(rr_intervals) if len(rr_intervals) > 0 else np.nan,
        'std_rr_interval_ms': np.std(rr_intervals) if len(rr_intervals) > 0 else np.nan
    })

subject_1_rr_interval_df = pd.DataFrame(subject_1_rr_interval_stats)

print(subject_1_rr_interval_df.head())

   subject_id  avg_rr_interval_ms  std_rr_interval_ms
0           1          146.048732          130.102519


In [ ]:

fig = px.line(x=subject_1_df['time'], y=subject_1_df['ppg_signal'], title='Subject 1 PPG Signal Over Time')
fig.add_scatter(x=subject_1_df['time'].iloc[peaks], y=subject_1_df['ppg_signal'].iloc[peaks], mode='markers', name='Peaks')
fig.show()

In [ ]:


fig = px.line(x=subject_1_df['time'], y=subject_1_df['ppg_signal'], title='Subject 1 PPG Signal Over Time')
fig.add_scatter(x=subject_1_df['time'].iloc[peaks], y=subject_1_df['ppg_signal'].iloc[peaks], mode='markers', name='Peaks')
fig.add_scatter(x=subject_1_df['time'].iloc[peaks + 1], y=subject_1_df['ppg_signal'].iloc[peaks + 1], mode='markers', name='Velly')
fig.show()

In [ ]:


time_diff = np.diff(subject_1_df['time'].values[peaks]) * 60
print(time_diff)

[ 9.60160027  3.00050008  2.40040007  1.20020003 21.00350058  1.20020003
 11.40190032  1.20020003 12.00200033 12.60210035 12.60210035  1.80030005
 23.40390065  1.20020003  9.60160027  1.80030005  1.20020003 23.40390065
 13.20220037  1.20020003 23.40390065 12.00200033  1.80030005  1.20020003
  9.60160027 13.20220037 10.20170028  1.20020003  1.80030005  1.20020003
  1.20020003  9.60160027 12.00200033 13.20220037  1.20020003  1.20020003
  9.00150025 13.20220037 12.00200033  1.20020003  1.80030005 10.8018003
 12.60210035 10.20170028  3.00050008 24.00400067  1.20020003 11.40190032
  1.20020003  1.20020003 22.80380063 10.8018003   1.80030005  1.20020003
 23.40390065 11.40190032  1.80030005  1.20020003 24.00400067 10.20170028
  1.80030005  1.20020003  1.20020003 22.20370062  1.20020003 12.60210035
 24.60410068 10.8018003   3.00050008 24.00400067 10.20170028  1.20020003
  1.80030005  1.20020003 23.40390065 12.00200033  1.20020003 22.80380063
  1.20020003  1.20020003 10.8018003   1.80030005  1.

In [ ]:


fig = px.line(x=subject_1_df['time'], y=subject_1_df['ppg_signal'], title='Subject 1 PPG Signal Over Time')
fig.add_scatter(x=subject_1_df['time'].iloc[peaks], y=subject_1_df['ppg_signal'].iloc[peaks], mode='markers', name='Peaks')
fig.add_scatter(x=subject_1_df['time'].iloc[peaks + 1], y=subject_1_df['ppg_signal'].iloc[peaks + 1], mode='markers', name='Velly')
fig.add_scatter(x=subject_1_df['time'].iloc[peaks + 2], y=subject_1_df['ppg_signal'].iloc[peaks + 2], mode='markers', name='Velly')
fig.show()

In [ ]:

rr_intervals_ms = np.diff(subject_1_df['time'].iloc[peaks].values) * 1000  # assuming time in seconds

subject_1_df['rr_interval_ms'] = np.nan
subject_1_df.loc[peaks[1:], 'rr_interval_ms'] = rr_intervals_ms

subject_1_df['HR_bpm'] = 60 * 1000 / subject_1_df['rr_interval_ms']  # RR in ms

avg_hr = subject_1_df['HR_bpm'].mean(skipna=True)

print(f"Average Heart Rate (BPM): {avg_hr:.2f} bpm")

Average Heart Rate (BPM): 95.58 bpm


In [ ]:

valid_rr_ms = subject_1_df['rr_interval_ms'].dropna()
valid_rr_minutes = valid_rr_ms / 60000  
sdnn_minutes = np.std(valid_rr_minutes)

print(f"HRV (SDNN): {sdnn_minutes:.6f} minutes")


HRV (SDNN): 0.000147 minutes


same way train from 1 to 18 and subjec 19 and 20 is for tesiing add feature if rr interval is this then hr will be that

In [ ]:
#training the whole model
import pandas as pd
import numpy as np
import os
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

data_folder = r"C:\Users\DELL\Desktop\vocational traning\project 1"  # Correct path to your subject files
subject_features = []


for i in range(1, 21):  
    file_path = os.path.join(data_folder, f"subject_{i}.csv")
    
    
    df = pd.read_csv(file_path)
    

    peaks, _ = find_peaks(df['ppg_signal'], distance=30)  #

    
    rr_intervals = np.diff(df['time'].iloc[peaks].values) * 1000 
    
    if len(rr_intervals) == 0:
        continue  
    
    
    hr_values = 60 * 1000 / rr_intervals
    avg_hr = np.mean(hr_values)


    sdnn = np.std(rr_intervals)


    subject_features.append({
        'subject_id': i,
        'avg_hr': avg_hr,
        'hrv_sdnn': sdnn
    })


features_df = pd.DataFrame(subject_features)


master_df = pd.read_csv(r'C:\Users\DELL\Desktop\vocational traning\project 1\master_data (1).csv')

merged_df = pd.merge(features_df, master_df, on='subject_id')


X = merged_df[['avg_hr', 'hrv_sdnn']]
y = merged_df['hrv']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.36, random_state=30)


model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModel Evaluation:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")



Model Evaluation:
Mean Squared Error: 221.8245
R² Score: 0.0145


In [ ]:
%pip install neurokit2 pywt

import pandas as pd
import numpy as np
from scipy.signal import find_peaks, welch
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import make_scorer, r2_score
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_regression
import neurokit2 as nk  # For advanced HRV features

# Enhanced Feature Extraction
def extract_features(df):
    features = {}
    signal = df['ppg_signal'].values
    
    # 1. Time Domain Features
    peaks, _ = find_peaks(signal, distance=30, prominence=0.5)
    rr_intervals = np.diff(df['time'].iloc[peaks]) * 1000  # in ms
    
    # Filter physiologically plausible RR intervals
    rr_intervals = rr_intervals[(rr_intervals > 300) & (rr_intervals < 2000)]
    
    if len(rr_intervals) < 5:  # Need minimum 5 beats for HRV
        return None
    
    # Neurokit for advanced HRV metrics
    try:
        hrv_features = nk.hrv_time(rr_intervals, sampling_rate=100)
        features.update(hrv_features)
    except:
        pass
    
    # 2. Frequency Domain Features
    f, psd = welch(signal, fs=100, nperseg=256)
    for band, (low, high) in {'lf':(0.04,0.15), 'hf':(0.15,0.4)}.items():
        mask = (f >= low) & (f <= high)
        features[f'ppg_{band}_power'] = np.trapz(psd[mask], f[mask])
    
    # 3. Non-linear Features
    features['ppg_entropy'] = nk.entropy_sample(signal)
    
    return features

# Data Processing Pipeline
all_features = []
for subj_id in df['subject_id'].unique():
    subj_df = df[df['subject_id'] == subj_id]
    feats = extract_features(subj_df)
    if feats:
        feats['subject_id'] = subj_id
        all_features.append(feats)

features_df = pd.DataFrame(all_features)
merged_df = pd.merge(features_df, master_df, on='subject_id')

# Feature Selection
X = merged_df.drop(['subject_id', 'hrv'], axis=1)
y = merged_df['hrv']

# Remove low-variance and highly correlated features
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]
X = X.drop(to_drop, axis=1)

# Time-series aware validation
tscv = TimeSeriesSplit(n_splits=5)
model = Pipeline([
    ('scaler', RobustScaler()),
    ('regressor', RandomForestRegressor(
        n_estimators=200,
        max_depth=5,
        min_samples_leaf=3,
        random_state=42
    ))
])

# Evaluation
cv_scores = cross_val_score(model, X, y, cv=tscv, 
                          scoring=make_scorer(r2_score))
print(f"Cross-validated R²: {np.mean(cv_scores):.2f} (±{np.std(cv_scores):.2f})")

# Feature Importance Analysis
model.fit(X, y)
importances = pd.Series(model.named_steps['regressor'].feature_importances_, 
                       index=X.columns)
print(importances.sort_values(ascending=False).head(10))

ERROR: Could not find a version that satisfies the requirement pywt (from versions: none)
ERROR: No matching distribution found for pywt

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'pywt'

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# 1. Enhanced Feature Engineering
def extract_ppg_features(df):
    features = {}
    
    # Basic signal stats
    features['ppg_mean'] = df['ppg_signal'].mean()
    features['ppg_std'] = df['ppg_signal'].std()
    
    # Peak detection with adaptive thresholds
    peaks, properties = find_peaks(df['ppg_signal'], 
                                  distance=30,
                                  prominence=(0.5, None))
    
    if len(peaks) < 4:
        return None  # Not enough peaks
    
    # RR intervals features
    rr_intervals = np.diff(df['time'].iloc[peaks]) * 1000
    rr_intervals = rr_intervals[(rr_intervals > 300) & (rr_intervals < 2000)]  # physiological range
    
    if len(rr_intervals) < 3:
        return None
        
    features.update({
        'hr_mean': 60000 / np.mean(rr_intervals),
        'hr_std': np.std(60000 / rr_intervals),
        'sdnn': np.std(rr_intervals),
        'rmssd': np.sqrt(np.mean(np.square(np.diff(rr_intervals)))),
        'nn50': np.sum(np.abs(np.diff(rr_intervals)) > 50),
        'pnn50': np.mean(np.abs(np.diff(rr_intervals)) > 50) * 100,
    })
    
    # Frequency domain features (very basic approximation)
    fft_values = np.abs(np.fft.rfft(df['ppg_signal']))
    features['ppg_fft_peak'] = np.max(fft_values[1:])  # exclude DC component
    
    return features

# 2. Data Processing Pipeline
subject_features = []
for i in range(1, 21):
    try:
        df = pd.read_csv(f"subject_{i}.csv")
        features = extract_ppg_features(df)
        if features:
            features['subject_id'] = i
            subject_features.append(features)
    except:
        continue

features_df = pd.DataFrame(subject_features)
# Use the already loaded master_df from previous cells
merged_df = pd.merge(features_df, master_df, on='subject_id')

# 3. Feature Selection
X = merged_df.drop(['subject_id', 'hrv'], axis=1)
y = merged_df['hrv']

# Remove highly correlated features
corr_matrix = X.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X = X.drop(to_drop, axis=1)

# 4. Modeling with Cross-Validation
model = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')

print("Cross-Validation R² Scores:", cv_scores)
print(f"Mean CV R²: {np.mean(cv_scores):.2f} (±{np.std(cv_scores):.2f})")

# 5. Final Evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=30)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("\nTest Set Performance:")
print(f"R²: {r2_score(y_test, y_pred):.2f}")
print(f"MSE: {mean_squared_error(y_test, y_pred):.2f}")



Cross-Validation R² Scores: [-2.81428356 -6.79735027 -0.68059624 -4.60905905 -0.67652107]
Mean CV R²: -3.12 (±2.36)

Test Set Performance:
R²: 0.19
MSE: 62.55
